In [1]:
import pandas as pd
import geopandas as gpd

/home/data/work/Fatemeh/DEiA-2-main/.direnv/python-3.10.1/lib/python3.10/site-packages/geopandas/_compat.py:111: UserWarning: The Shapely GEOS version (3.10.2-CAPI-1.16.0) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.
  warnings.warn(


In [2]:
facilities = pd.read_csv('nl_osm_locations.csv')
facilities = gpd.GeoDataFrame(
    facilities, geometry=gpd.points_from_xy(facilities['lon'], facilities['lat'])
).drop(columns=['lat', 'lon'])
facilities

,Unnamed: 0,id,name,addr:postcode,type,geometry
0,0,node/290848021,PSZ Het Hummelhonk,NaN,nl_kindergarten,POINT (3.79618 51.22780)
1,1,node/299240246,Mareland,NaN,nl_kindergarten,POINT (4.49153 52.16506)
2,2,node/304623674,Angela's Kroost,NaN,nl_kindergarten,POINT (4.46853 52.19076)
3,3,node/304626605,Het kindercircus,NaN,nl_kindergarten,POINT (4.47969 52.19694)
4,4,node/305501081,De Hobbit,5663ST,nl_kindergarten,POINT (5.54686 51.41163)
...,...,...,...,...,...,...
5445,5445,node/9418182063,Fit-Now!,9231DX,nl_gyms,POINT (6.17622 53.17208)
5446,5446,node/9452429013,Vikings gym,6431DH,nl_gyms,POINT (5.92217 50.93254)
5447,5447,node/9452717476,CrossFit Hilversum - CrossFit & Personal Training,1216SL,nl_gyms,POINT (5.14585 52.21238)
5448,5448,node/9495450226,Dudok Gym,1217ET,nl_gyms,POINT (5.16862 52.22747)


In [3]:
neighborhoods = pd.read_csv('output_list.csv')
neighborhoods = gpd.GeoDataFrame(
    neighborhoods, geometry=gpd.points_from_xy(neighborhoods['longitude'], neighborhoods['latitude'])
).drop(columns=['latitude', 'longitude'])
neighborhoods

,neighbourhood,municipality,play_demand,a_00_14,geometry
0,Appingedam-Oost,Appingedam,7,900,POINT (6.85937 53.30707)
1,Hoogkerk-Zuid,Groningen,5,965,POINT (6.49227 53.20587)
2,Beijum-West,Groningen,6,1005,POINT (6.58710 53.24935)
3,Beijum-Oost,Groningen,6,1175,POINT (6.60101 53.24858)
4,Haren,Groningen,6,1485,POINT (6.58583 53.17099)
...,...,...,...,...,...
486,Numansdorp Woonkern,Hoeksche Waard,6,1110,POINT (4.43821 51.73477)
487,Bedum,Het Hogeland,6,1205,POINT (6.59983 53.30168)
488,Marum,Westerkwartier,5,910,POINT (6.26733 53.14625)
489,Zuidhorn,Westerkwartier,6,1475,POINT (6.40810 53.24315)


In [4]:
def join_nearest_type(type_):
    return gpd.sjoin_nearest(neighborhoods, facilities.loc[facilities['type'] == type_], distance_col='distance')

In [5]:
def join_nearest_types(types):
    dfs = [join_nearest_type(t) for t in types]
    return pd.concat(dfs, axis=0, ignore_index=True)

In [6]:
facility_types = list(facilities['type'].unique())
facility_types

['nl_kindergarten',
 'nl_childcare',
 'nl_school',
 'nl_college',
 'nl_university',
 'nl_nursing_house',
 'nl_park',
 'nl_gyms']

In [7]:
joined = join_nearest_types(facility_types)
joined

,neighbourhood,municipality,play_demand,a_00_14,geometry,index_right,Unnamed: 0,id,name,addr:postcode,type,distance
0,Appingedam-Oost,Appingedam,7,900,POINT (6.85937 53.30707),368,368,node/2835405709,De Wieke,9903ET,nl_kindergarten,0.012260
1,Hoogkerk-Zuid,Groningen,5,965,POINT (6.49227 53.20587),262,262,node/2738116910,SKSG Lancelot,9746BJ,nl_kindergarten,0.028856
2,Beijum-West,Groningen,6,1005,POINT (6.58710 53.24935),281,281,node/2744239619,Hamertje Tik,9737TB,nl_kindergarten,0.001756
3,Beijum-Oost,Groningen,6,1175,POINT (6.60101 53.24858),489,489,node/3764445890,Hamertje Tik,NaN,nl_kindergarten,0.000766
4,Haren,Groningen,6,1485,POINT (6.58583 53.17099),269,269,node/2740107533,Het Helpernest,9721AJ,nl_kindergarten,0.026842
...,...,...,...,...,...,...,...,...,...,...,...,...
3923,Strijen Woonkern,Hoeksche Waard,6,950,POINT (4.55392 51.74468),4845,4845,node/2767239516,Sport Boutique 32,3291CH,nl_gyms,0.002744
3924,'s-Gravendeel Woonkern,Hoeksche Waard,6,1315,POINT (4.61809 51.77928),5143,5143,node/4452537668,Fit For Free,3331EE,nl_gyms,0.041082
3925,Numansdorp Woonkern,Hoeksche Waard,6,1110,POINT (4.43821 51.73477),4864,4864,node/2776771981,Sportcentrum Numansdorp,3281NC,nl_gyms,0.014673
3926,Bedum,Het Hogeland,6,1205,POINT (6.59983 53.30168),4826,4826,node/2754843890,GO4fitness,9781AK,nl_gyms,0.008809


In [8]:
joined['distance'] = round(joined['distance'] * 110.486, 2)
joined

,neighbourhood,municipality,play_demand,a_00_14,geometry,index_right,Unnamed: 0,id,name,addr:postcode,type,distance
0,Appingedam-Oost,Appingedam,7,900,POINT (6.85937 53.30707),368,368,node/2835405709,De Wieke,9903ET,nl_kindergarten,1.35
1,Hoogkerk-Zuid,Groningen,5,965,POINT (6.49227 53.20587),262,262,node/2738116910,SKSG Lancelot,9746BJ,nl_kindergarten,3.19
2,Beijum-West,Groningen,6,1005,POINT (6.58710 53.24935),281,281,node/2744239619,Hamertje Tik,9737TB,nl_kindergarten,0.19
3,Beijum-Oost,Groningen,6,1175,POINT (6.60101 53.24858),489,489,node/3764445890,Hamertje Tik,NaN,nl_kindergarten,0.08
4,Haren,Groningen,6,1485,POINT (6.58583 53.17099),269,269,node/2740107533,Het Helpernest,9721AJ,nl_kindergarten,2.97
...,...,...,...,...,...,...,...,...,...,...,...,...
3923,Strijen Woonkern,Hoeksche Waard,6,950,POINT (4.55392 51.74468),4845,4845,node/2767239516,Sport Boutique 32,3291CH,nl_gyms,0.30
3924,'s-Gravendeel Woonkern,Hoeksche Waard,6,1315,POINT (4.61809 51.77928),5143,5143,node/4452537668,Fit For Free,3331EE,nl_gyms,4.54
3925,Numansdorp Woonkern,Hoeksche Waard,6,1110,POINT (4.43821 51.73477),4864,4864,node/2776771981,Sportcentrum Numansdorp,3281NC,nl_gyms,1.62
3926,Bedum,Het Hogeland,6,1205,POINT (6.59983 53.30168),4826,4826,node/2754843890,GO4fitness,9781AK,nl_gyms,0.97


In [9]:
joined.loc[joined['neighbourhood'] == 'Haren']

,neighbourhood,municipality,play_demand,a_00_14,geometry,index_right,Unnamed: 0,id,name,addr:postcode,type,distance
4,Haren,Groningen,6,1485,POINT (6.58583 53.17099),269,269,node/2740107533,Het Helpernest,9721AJ,nl_kindergarten,2.97
496,Haren,Groningen,6,1485,POINT (6.58583 53.17099),917,917,node/2748274225,Kinderopvang Groningen,9725AC,nl_childcare,4.35
986,Haren,Groningen,6,1485,POINT (6.58583 53.17099),1819,1819,node/1357031613,Mytylschool Prins Johan Friso,NaN,nl_school,2.01
1483,Haren,Groningen,6,1485,POINT (6.58583 53.17099),4235,4235,node/649061908,Noorderpoort College Euroborg,9723ZS,nl_college,3.99
1971,Haren,Groningen,6,1485,POINT (6.58583 53.17099),4467,4467,node/4681578917,Hanzehogeschool (Meerwold),NaN,nl_university,3.06
2465,Haren,Groningen,6,1485,POINT (6.58583 53.17099),4499,4499,node/1461155399,Heymanscentrum Neerwolde,9728HD,nl_nursing_house,3.83
2950,Haren,Groningen,6,1485,POINT (6.58583 53.17099),4599,4599,node/1910188455,Hoornse Schans,NaN,nl_park,4.62
3441,Haren,Groningen,6,1485,POINT (6.58583 53.17099),4801,4801,node/2740108333,Dreamfit,9721AJ,nl_gyms,2.93


In [10]:
facilities.loc[facilities['name'] == 'Dreamfit']

,Unnamed: 0,id,name,addr:postcode,type,geometry
4801,4801,node/2740108333,Dreamfit,9721AJ,nl_gyms,POINT (6.58053 53.19697)
